In [1]:
import hashlib
import sys
import os
import pandas as pd

In [2]:
DEBUG = False

# Função para adicionar o caminho do módulo 'utils' ao sys.path
utils_path = os.path.abspath("../") if DEBUG else os.path.abspath("../../")
if utils_path not in sys.path:
    sys.path.append(utils_path)

In [ ]:
from utils.auxiliary_functions.all_auxiliary_functions import (
    extrair_dados_database,
    carregar_dados_database,
)

In [1]:
def gerar_periodos(df):
    """
    Gera uma lista de períodos entre o ano mínimo e máximo encontrados
    na coluna 'periodoIngressoUFRJ' do DataFrame.

    Args:
        df (DataFrame): DataFrame contendo a coluna 'periodoIngressoUFRJ'.

    Returns:
        DataFrame: DataFrame com os períodos gerados.
    """
    min_periodo = int(df["periodoIngressoUFRJ"].min().split("/")[0])
    max_periodo = int(df["periodoIngressoUFRJ"].max().split("/")[0])

    anos = range(min_periodo, max_periodo + 1)
    semestres = [0, 1, 2, 3]

    data = [
        [ano, semestre, f"{ano}/{semestre}"] for ano in anos for semestre in semestres
    ]

    return pd.DataFrame(data, columns=["VL_ANO", "VL_SEMESTRE", "DS_PERIODO"])

In [ ]:
df_silver = extrair_dados_database("alunos/alunos.parquet", "silver")

In [ ]:
# Colunas da dimensão PERÍODO
colunas_periodo = ["SK_D_PERIODO", "VL_ANO", "VL_SEMESTRE", "DS_PERIODO"]

# Geração dos períodos
d_periodo = gerar_periodos(df_silver)

# Criação do SK_D_PERIODO usando SHA-256
d_periodo["SK_D_PERIODO"] = d_periodo.apply(
    lambda x: hashlib.sha256(str(x["DS_PERIODO"]).encode()).hexdigest(), axis=1
)

# Reordenação das colunas
d_periodo = d_periodo.reindex(columns=colunas_periodo)

In [ ]:
# Salvar dados na camada Gold
carregar_dados_database(d_periodo, "d_periodo", "gold", "parquet")